# Hyperparameter Optimization

Inspired by https://www.tensorflow.org/tensorboard/r2/hyperparameter_tuning_with_hparams

In [ ]:
import shutil
log_dir = "/tmp/tensorboard/hparam_tuning/"
shutil.rmtree(log_dir)

In [ ]:
import datetime
import tensorflow as tf

from tensorboard.plugins.hparams import api_pb2
from tensorboard.plugins.hparams import summary as hparams_summary

import numpy as np
from scipy.stats.distributions import expon, uniform, randint
from sklearn.model_selection import ParameterGrid, ParameterSampler
%matplotlib inline
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D
from tensorflow.keras.optimizers import Adam

In [ ]:
(X_train, y_train),(X_test, y_test) = cifar10.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0

In [ ]:
def train_test_model(hparams):
    model = Sequential()
    for i in range(hparams['n_conv_layers']):
        model.add(Conv2D(hparams['n_conv_filters'], (3,3)))
    model.add(Flatten())
    model.add(Dense(hparams['n_units'], activation='relu'))
    model.add(Dropout(hparams['dropout_rate']))
    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer=Adam(lr=hparams['learning_rate']),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    model.fit(X_train, y_train,
              epochs=hparams['epochs'],
              batch_size=hparams['batch_size'])
    _, accuracy = model.evaluate(X_test, y_test)
    return accuracy

In [ ]:
hp_ranges = {
    'n_conv_layers': [1],
    'n_conv_filters': [16],
    'n_units': [16, 32],
    'dropout_rate': [0.1, 0.2],
    'batch_size': [32],
    'learning_rate':[0.001],
    'epochs': [1]
}

In [ ]:
def run(run_dir, hparams):
    writer = tf.summary.create_file_writer(run_dir)
    summary_start = hparams_summary.session_start_pb(hparams=hparams)

    with writer.as_default():
        accuracy = train_test_model(hparams)
        summary_end = hparams_summary.session_end_pb(api_pb2.STATUS_SUCCESS)
          
        tf.summary.scalar('accuracy', accuracy, step=1, description="The accuracy")
        tf.summary.import_event(tf.compat.v1.Event(summary=summary_start).SerializeToString())
        tf.summary.import_event(tf.compat.v1.Event(summary=summary_end).SerializeToString())

## Grid Search

In [ ]:
for hparams in ParameterGrid(hp_ranges):
    print(hparams)

In [ ]:
session_num = 0

for hparams in ParameterGrid(hp_ranges):
    print('--- Running training session %d' % (session_num + 1))
    print(hparams)
    run_name = "run-%d" % session_num
    run(log_dir + run_name, hparams)
    session_num += 1

Now check Tensorboard

![hparams.png](../assets/hparams.png)

## Exercise: 

Implement a Random Search with a much larger set of possibilities using `sklearn.model_selection.ParameterSampler`

In [ ]:
hp_ranges = {
    'n_conv_layers': randint(1, 3),
    'n_conv_filters': [32, 64, 128],
    'n_units': [16, 32, 64, 128],
    'dropout_rate': uniform(loc=0.0, scale=0.6),
    'batch_size': [8, 16, 32, 64, 128],
    'learning_rate': [0.03, 0.001],
    'epochs': [2]
}

In [ ]:
def print_dict(d):
    for k, v in d.items():
        print('  {:<20}: {}'.format(k, v))

In [ ]:
hp_sets = ParameterSampler(hp_ranges, n_iter=2, random_state=0)

for i, hp_set in enumerate(hp_sets):
    print()
    print("Hyperparameter Set {}:".format(i))
    print_dict(hp_set)

In [ ]:
session_num = 0

for hparams in ParameterSampler(hp_ranges, n_iter=30, random_state=0):
    print('--- Running training session %d' % (session_num + 1))
    print(hparams)
    run_name = "run-%d" % session_num
    run(log_dir + run_name, hparams)
    session_num += 1